# OpenAI APIs - Vision

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/guides/vision).
This tutorial covers the vision APIs for vision language models.

SGLang supports various vision language models such as Llama 3.2, LLaVA-OneVision, Qwen2.5-VL, Gemma3 and [more](https://docs.sglang.ai/supported_models/vision_language_models): 
- [meta-llama/Llama-3.2-11B-Vision-Instruct](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct)  
- [lmms-lab/llava-onevision-qwen2-72b-ov-chat](https://huggingface.co/lmms-lab/llava-onevision-qwen2-72b-ov-chat)  
- [Qwen/Qwen2.5-VL-7B-Instruct](https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct)
- [google/gemma-3-4b-it](https://huggingface.co/google/gemma-3-4b-it)
- [openbmb/MiniCPM-V](https://huggingface.co/openbmb/MiniCPM-V)
- [deepseek-ai/deepseek-vl2](https://huggingface.co/deepseek-ai/deepseek-vl2)

As an alternative to the OpenAI API, you can also use the [SGLang offline engine](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py).

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

**Remember to add** `--chat-template` **for example** `--chat-template=qwen2-vl` **to specify the [vision chat template](https://docs.sglang.ai/backend/openai_api_vision.html#Chat-Template), otherwise, the server will only support text (images won’t be passed in), which can lead to degraded performance.**

We need to specify `--chat-template` for vision language models because the chat template provided in Hugging Face tokenizer only supports text.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

vision_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Qwen/Qwen2.5-VL-7B-Instruct \
    --chat-template=qwen2-vl
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-05-07 16:11:46] server_args=ServerArgs(model_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-VL-7B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, enable_tokenizer_batch_encode=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen2.5-VL-7B-Instruct', chat_template='qwen2-vl', completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=39608, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=226021593, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu

[2025-05-07 16:11:48] Use chat template for the OpenAI-compatible API server: qwen2-vl


[2025-05-07 16:11:58] Attention backend not set. Use flashinfer backend by default.
[2025-05-07 16:11:58] Automatically reduce --mem-fraction-static to 0.792 because this is a multimodal model.
[2025-05-07 16:11:58] Automatically turn off --chunked-prefill-size for multimodal model.
[2025-05-07 16:11:58] Init torch distributed begin.


[2025-05-07 16:11:59] Init torch distributed ends. mem usage=0.00 GB
[2025-05-07 16:11:59] Load weight begin. avail mem=48.30 GB


[2025-05-07 16:11:59] Ignore import error when loading sglang.srt.models.llama4. 


...

[2025-05-07 16:12:06] INFO:     Started server process [2176586]
[2025-05-07 16:12:06] INFO:     Waiting for application startup.
[2025-05-07 16:12:06] INFO:     Application startup complete.
[2025-05-07 16:12:06] INFO:     Uvicorn running on http://127.0.0.1:39608 (Press CTRL+C to quit)


[2025-05-07 16:12:07] INFO:     127.0.0.1:47484 - "GET /v1/models HTTP/1.1" 200 OK


[2025-05-07 16:12:07] INFO:     127.0.0.1:47496 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-05-07 16:12:07] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


2025-05-07 16:12:09,673 - INFO - flashinfer.jit: Loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False
2025-05-07 16:12:09,799 - INFO - flashinfer.jit: Finished loading JIT ops: batch_prefill_with_kv_cache_dtype_q_bf16_dtype_kv_bf16_dtype_o_bf16_dtype_idx_i32_head_dim_qk_128_head_dim_vo_128_posenc_0_use_swa_False_use_logits_cap_False_f16qk_False


[2025-05-07 16:12:10] INFO:     127.0.0.1:47498 - "POST /generate HTTP/1.1" 200 OK
[2025-05-07 16:12:10] The server is fired up and ready to roll!


## Using cURL

Once the server is up, you can send test requests using curl or requests.

In [2]:
import subprocess

curl_command = f"""
curl -s http://localhost:{port}/v1/chat/completions \\
  -d '{{
    "model": "Qwen/Qwen2.5-VL-7B-Instruct",
    "messages": [
      {{
        "role": "user",
        "content": [
          {{
            "type": "text",
            "text": "What’s in this image?"
          }},
          {{
            "type": "image_url",
            "image_url": {{
              "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
            }}
          }}
        ]
      }}
    ],
    "max_tokens": 300
  }}'
"""

response = subprocess.check_output(curl_command, shell=True).decode()
print_highlight(response)


response = subprocess.check_output(curl_command, shell=True).decode()
print_highlight(response)

[2025-05-07 16:12:13] Prefill batch. #new-seq: 1, #new-token: 307, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-07 16:12:13] Decode batch. #running-req: 1, #token: 340, token usage: 0.02, gen throughput (token/s): 5.00, #queue-req: 0


[2025-05-07 16:12:14] Decode batch. #running-req: 1, #token: 380, token usage: 0.02, gen throughput (token/s): 60.51, #queue-req: 0


[2025-05-07 16:12:14] INFO:     127.0.0.1:47512 - "POST /v1/chat/completions HTTP/1.1" 200 OK


[2025-05-07 16:12:15] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 306, token usage: 0.01, #running-req: 0, #queue-req: 0


[2025-05-07 16:12:15] Decode batch. #running-req: 1, #token: 333, token usage: 0.02, gen throughput (token/s): 35.39, #queue-req: 0


[2025-05-07 16:12:16] Decode batch. #running-req: 1, #token: 373, token usage: 0.02, gen throughput (token/s): 61.30, #queue-req: 0
[2025-05-07 16:12:16] INFO:     127.0.0.1:47520 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using Python Requests

In [3]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "Qwen/Qwen2.5-VL-7B-Instruct",
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
                    },
                },
            ],
        }
    ],
    "max_tokens": 300,
}

response = requests.post(url, json=data)
print_highlight(response.text)

[2025-05-07 16:12:16] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 306, token usage: 0.01, #running-req: 0, #queue-req: 0


[2025-05-07 16:12:17] Decode batch. #running-req: 1, #token: 338, token usage: 0.02, gen throughput (token/s): 43.91, #queue-req: 0


[2025-05-07 16:12:17] Decode batch. #running-req: 1, #token: 378, token usage: 0.02, gen throughput (token/s): 61.60, #queue-req: 0
[2025-05-07 16:12:18] INFO:     127.0.0.1:47536 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using OpenAI Python Client

In [4]:
from openai import OpenAI

client = OpenAI(base_url=f"http://localhost:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="Qwen/Qwen2.5-VL-7B-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print_highlight(response.choices[0].message.content)

[2025-05-07 16:12:18] Prefill batch. #new-seq: 1, #new-token: 292, #cached-token: 15, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-07 16:12:19] Decode batch. #running-req: 1, #token: 336, token usage: 0.02, gen throughput (token/s): 37.42, #queue-req: 0


[2025-05-07 16:12:19] Decode batch. #running-req: 1, #token: 376, token usage: 0.02, gen throughput (token/s): 61.61, #queue-req: 0


[2025-05-07 16:12:19] INFO:     127.0.0.1:38244 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Multiple-Image Inputs

The server also supports multiple images and interleaved text and images if the model supports it.

In [5]:
from openai import OpenAI

client = OpenAI(base_url=f"http://localhost:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="Qwen/Qwen2.5-VL-7B-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true",
                    },
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://raw.githubusercontent.com/sgl-project/sglang/main/assets/logo.png",
                    },
                },
                {
                    "type": "text",
                    "text": "I have two very different images. They are not related at all. "
                    "Please describe the first image in one sentence, and then describe the second image in another sentence.",
                },
            ],
        }
    ],
    temperature=0,
)

print_highlight(response.choices[0].message.content)

[2025-05-07 16:12:20] Prefill batch. #new-seq: 1, #new-token: 2532, #cached-token: 14, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-05-07 16:12:21] Decode batch. #running-req: 1, #token: 2567, token usage: 0.13, gen throughput (token/s): 17.85, #queue-req: 0


[2025-05-07 16:12:22] INFO:     127.0.0.1:38260 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [6]:
terminate_process(vision_process)

[2025-05-07 16:12:22] Child process unexpectedly failed with an exit code 9. pid=2177013
[2025-05-07 16:12:22] Child process unexpectedly failed with an exit code 9. pid=2177010


## Chat Template

As mentioned before, if you do not specify a vision model's `--chat-template`, the server uses Hugging Face's default template, which only supports text.

We list popular vision models with their chat templates:

- [meta-llama/Llama-3.2-Vision](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct) uses `llama_3_vision`.
- [Qwen/Qwen2.5-VL-7B-Instruct](https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct) uses `qwen2-vl`.
- [google/gemma-3-4b-it](https://huggingface.co/google/gemma-3-4b-it) uses `gemma-it`.
- [openbmb/MiniCPM-V](https://huggingface.co/openbmb/MiniCPM-V) uses `minicpmv`.
- [deepseek-ai/deepseek-vl2](https://huggingface.co/deepseek-ai/deepseek-vl2) uses `deepseek-vl2`.
- [LlaVA-OneVision](https://huggingface.co/lmms-lab/llava-onevision-qwen2-7b-ov) uses `chatml-llava`.
- [LLaVA-NeXT](https://huggingface.co/collections/lmms-lab/llava-next-6623288e2d61edba3ddbf5ff) uses `chatml-llava`.
- [Llama3-LLaVA-NeXT](https://huggingface.co/lmms-lab/llama3-llava-next-8b) uses `llava_llama_3`.
- [LLaVA-v1.5 / 1.6](https://huggingface.co/liuhaotian/llava-v1.6-34b) uses `vicuna_v1.1`.